In [1]:
import pandas as pd
import numpy as np
import re
import nltk

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ajite\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ajite\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ajite\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
data = pd.read_csv("./dataset/amazon_reviews.csv")
data.head()

,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0,0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0,0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0,0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31
3,3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5,0,0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31
4,4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5,0,0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31


In [6]:
data = data.loc[:,['review_body',"star_rating"]]
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)
data.head()

,review_body,star_rating
0,so beautiful even tho clearly not high end ......,5
1,"Great product.. I got this set for my mother, ...",5
2,Exactly as pictured and my daughter's friend l...,5
3,Love it. Fits great. Super comfortable and nea...,5
4,Got this as a Mother's Day gift for my Mom and...,5


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99945 entries, 0 to 99944
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_body  99945 non-null  object
 1   star_rating  99945 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [7]:
## The reviews with star rating 4 , 5 are labelled as positive reviews and 1,2 are labelled as negative reviews
## Remove the reviews with star rating with 3 as they are considered as neutral

In [8]:
data.star_rating  = data.star_rating.astype(int)
data = data[data["star_rating"] != 3]
data['label']  = np.where(data.star_rating>=4,1,0) # 1-Positive , 0-negative
data.reset_index(inplace=True,drop=True)
data.head()

,review_body,star_rating,label
0,so beautiful even tho clearly not high end ......,5,1
1,"Great product.. I got this set for my mother, ...",5,1
2,Exactly as pictured and my daughter's friend l...,5,1
3,Love it. Fits great. Super comfortable and nea...,5,1
4,Got this as a Mother's Day gift for my Mom and...,5,1


In [9]:
data.star_rating.value_counts()

star_rating
5    62671
4    13919
1     9334
2     5512
Name: count, dtype: int64

In [10]:
data.label.value_counts()

label
1    76590
0    14846
Name: count, dtype: int64

In [11]:
## Preprocessing

In [12]:
import re
from bs4 import BeautifulSoup

In [13]:
data.loc[:,'preprocess'] = data.review_body.apply(lambda x : " ".join(x.lower() for x in str(x).split()))
data.head()

,review_body,star_rating,label,preprocess
0,so beautiful even tho clearly not high end ......,5,1,so beautiful even tho clearly not high end ......
1,"Great product.. I got this set for my mother, ...",5,1,"great product.. i got this set for my mother, ..."
2,Exactly as pictured and my daughter's friend l...,5,1,exactly as pictured and my daughter's friend l...
3,Love it. Fits great. Super comfortable and nea...,5,1,love it. fits great. super comfortable and nea...
4,Got this as a Mother's Day gift for my Mom and...,5,1,got this as a mother's day gift for my mom and...


In [14]:
## Remove html tags and URLS from the reviews
data["preprocess"] = data.preprocess.apply(lambda x :BeautifulSoup(x).get_text())
data.preprocess = data.preprocess.apply(lambda x:re.sub(r"http\S+","",x))

C:\Users\ajite\AppData\Local\Temp\ipykernel_9896\3766661073.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data["preprocess"] = data.preprocess.apply(lambda x :BeautifulSoup(x).get_text())
C:\Users\ajite\AppData\Local\Temp\ipykernel_9896\3766661073.py:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  data["preprocess"] = data.preprocess.apply(lambda x :BeautifulSoup(x).get_text())


In [15]:
def contractions(s):
    s = re.sub(r"won't", "will not",s)
    s = re.sub(r"would't", "would not",s)
    s = re.sub(r"could't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    s = re.sub(r"doesn't","does not",s)
    return s
data['preprocess']=data['preprocess'].apply(lambda x:contractions(x))

In [16]:
data

,review_body,star_rating,label,preprocess
0,so beautiful even tho clearly not high end ......,5,1,so beautiful even tho clearly not high end ......
1,"Great product.. I got this set for my mother, ...",5,1,"great product.. i got this set for my mother, ..."
2,Exactly as pictured and my daughter's friend l...,5,1,exactly as pictured and my daughter is friend ...
3,Love it. Fits great. Super comfortable and nea...,5,1,love it. fits great. super comfortable and nea...
4,Got this as a Mother's Day gift for my Mom and...,5,1,got this as a mother is day gift for my mom an...
...,...,...,...,...
91431,Delivery was quick. ...the beads are loose so ...,4,1,delivery was quick. ...the beads are loose so ...
91432,Tiny but cute! About 1/8th of an inch wide. Th...,5,1,tiny but cute! about 1/8th of an inch wide. th...
91433,So beautiful and stunning. There are cheapo p...,5,1,so beautiful and stunning. there are cheapo pi...
91434,The inlay is almost holographic. Very comforta...,5,1,the inlay is almost holographic. very comforta...


In [17]:
## Remove non-alpha characters
data.preprocess = data.preprocess.apply(lambda x : " ".join([re.sub('[^A-Za-z]+',"",x) for x in nltk.word_tokenize(x)]))
data

,review_body,star_rating,label,preprocess
0,so beautiful even tho clearly not high end ......,5,1,so beautiful even tho clearly not high end bu...
1,"Great product.. I got this set for my mother, ...",5,1,great product i got this set for my mother a...
2,Exactly as pictured and my daughter's friend l...,5,1,exactly as pictured and my daughter is friend ...
3,Love it. Fits great. Super comfortable and nea...,5,1,love it fits great super comfortable and nea...
4,Got this as a Mother's Day gift for my Mom and...,5,1,got this as a mother is day gift for my mom an...
...,...,...,...,...
91431,Delivery was quick. ...the beads are loose so ...,4,1,delivery was quick the beads are loose so u ...
91432,Tiny but cute! About 1/8th of an inch wide. Th...,5,1,tiny but cute about th of an inch wide the b...
91433,So beautiful and stunning. There are cheapo p...,5,1,so beautiful and stunning there are cheapo pi...
91434,The inlay is almost holographic. Very comforta...,5,1,the inlay is almost holographic very comforta...


In [18]:
## Remove extra spaces between the words

In [19]:
data.preprocess = data.preprocess.apply(lambda x: re.sub(" +",' ',x))

In [20]:
## REmove stopwords

In [21]:
from nltk.corpus import stopwords
stop = stopwords.words("english")
data.preprocess = data.preprocess.apply(lambda x:" ".join([x for x in x.split() if x not in stop]))

In [22]:
data.head()

,review_body,star_rating,label,preprocess
0,so beautiful even tho clearly not high end ......,5,1,beautiful even tho clearly high end super grea...
1,"Great product.. I got this set for my mother, ...",5,1,great product got set mother allergic bijoux p...
2,Exactly as pictured and my daughter's friend l...,5,1,exactly pictured daughter friend loved came pa...
3,Love it. Fits great. Super comfortable and nea...,5,1,love fits great super comfortable neat idea fi...
4,Got this as a Mother's Day gift for my Mom and...,5,1,got mother day gift mom loved really nice qual...


In [23]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data.preprocess = data.preprocess.apply(lambda x:" ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))
data.head()

,review_body,star_rating,label,preprocess
0,so beautiful even tho clearly not high end ......,5,1,beautiful even tho clearly high end super grea...
1,"Great product.. I got this set for my mother, ...",5,1,great product got set mother allergic bijou pr...
2,Exactly as pictured and my daughter's friend l...,5,1,exactly pictured daughter friend loved came pa...
3,Love it. Fits great. Super comfortable and nea...,5,1,love fit great super comfortable neat idea fis...
4,Got this as a Mother's Day gift for my Mom and...,5,1,got mother day gift mom loved really nice qual...


In [24]:
## Split the data into training and testing

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data.preprocess,data.label,test_size=0.25,random_state=30)
print("Train: ",X_train.shape,y_train.shape,"Test: ",(X_test.shape,y_test.shape))

Train:  (68577,) (68577,) Test:  ((22859,), (22859,))


In [26]:
## Feature extraction
## TF_Idf vectorizer

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

In [28]:
## Using SVC algorithm

In [29]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [30]:
clf.fit(tf_x_train,y_train)

C:\Users\ajite\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC(random_state=0)

In [31]:
y_test_pred = clf.predict(tf_x_test)
print(y_test_pred)

[1 1 1 ... 1 0 1]


In [32]:
## analyzing the results

In [33]:
from sklearn.metrics import classification_report
report=classification_report(y_test,y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.86      0.77      0.81      3756
           1       0.96      0.98      0.97     19103

    accuracy                           0.94     22859
   macro avg       0.91      0.87      0.89     22859
weighted avg       0.94      0.94      0.94     22859



In [34]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver="saga")
clf.fit(tf_x_train,y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [35]:
y_test_pred = clf.predict(tf_x_test)

In [36]:
from sklearn.metrics import classification_report
report=classification_report(y_test,y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.73      0.80      3756
           1       0.95      0.98      0.97     19103

    accuracy                           0.94     22859
   macro avg       0.92      0.86      0.88     22859
weighted avg       0.94      0.94      0.94     22859



In [167]:
example_text = 'Very bad product'
clf.predict(vectorizer.transform([example_text]))[0]

0

In [37]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [38]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [39]:
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)
data['tokenized'] = data['preprocess'].apply(tokenize_function)

In [40]:
from sklearn.model_selection import train_test_split

# Split the dataset
train_texts, temp_texts, train_labels, temp_labels = train_test_split(data['tokenized'].to_list(), data['label'].to_list(), test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

In [41]:
def convert_to_dicts(tokenized_texts):
    input_ids = [d['input_ids'] for d in tokenized_texts]
    attention_masks = [d['attention_mask'] for d in tokenized_texts]
    return {'input_ids': input_ids, 'attention_mask': attention_masks}

# Convert to lists of dictionaries
train_encodings = convert_to_dicts(train_texts)
val_encodings = convert_to_dicts(val_texts)
test_encodings = convert_to_dicts(test_texts)

# Create three dataset objects using the SentimentDataset
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [83]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [110]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary',zero_division=0
    )
    
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=30, 
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    warmup_steps=10, 
    weight_decay=0.1, 
    logging_dir='./logs',
    max_steps=30,  
    save_steps=2,  
    logging_steps=2,  
)

In [152]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None,zero_division=0
    )
    
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [ ]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()


Step,Training Loss
2,0.311200
4,0.321000
6,0.310800
8,0.214600
10,0.582000
12,0.258200
14,0.154300
16,0.162700
18,0.264700
20,0.271900


TrainOutput(global_step=30, training_loss=0.3049207960565885, metrics={'train_runtime': 534.7883, 'train_samples_per_second': 1.122, 'train_steps_per_second': 0.056, 'total_flos': 39466658304000.0, 'train_loss': 0.3049207960565885, 'epoch': 0.00937207122774133})

In [154]:
results = trainer.evaluate(test_dataset)

print("Evaluation Results:")
print(f"  - Loss: {results['eval_loss']:.4f}")
print(f"  - Runtime: {results['eval_runtime']:.2f} seconds")
print(f"  - Samples per Second: {results['eval_samples_per_second']:.2f}")
print(f"  - Steps per Second: {results['eval_steps_per_second']:.2f}")
print(f"  - Epoch: {results['epoch']:.4f}")
print(f"  - Accuracy: {results['eval_accuracy']:.4f}")
print(f"  - f1-score for class 0 and class 1: {results['eval_f1'][0]:.4f} & {results['eval_f1'][1]:.4f}")
print(f"  - recall for class 0 and class 1: {results['eval_recall'][0]:.4f} & {results['eval_recall'][1]:.4f} ")
print(f"  - precision for class 0 and class 1: {results['eval_precision'][0]:.4f} & {results['eval_precision'][0]:.4f} ")

Evaluation Results:
  - Loss: 0.2952
  - Runtime: 3809.23 seconds
  - Samples per Second: 3.60
  - Steps per Second: 0.18
  - Epoch: 0.0094
  - Accuracy: 0.8825
  - f1-score for class 0 and class 1: 0.5393 & 0.9327
  - recall for class 0 and class 1: 0.4288 & 0.9692 
  - precision for class 0 and class 1: 0.7265 & 0.7265 


In [159]:
model_save_path = "./model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./model\\tokenizer_config.json',
 './model\\special_tokens_map.json',
 './model\\vocab.txt',
 './model\\added_tokens.json')

In [163]:
def predict_sentiment(text):
    threshold=0.6
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    negative_prob = probs[0][0].item()
    positive_prob = probs[0][1].item()
    # print(positive_prob,negative_prob)
    
    if negative_prob >= 0.04:
        return 'negative'
    elif positive_prob >= threshold:
        return 'positive'


example_text = "poor performance"
predicted = predict_sentiment(example_text)
print(f"Predicted Sentiment: {predicted}")

Predicted Sentiment: negative
